# Sporkful Sandwiches

Sandwich data from a recent [Sporkful podcast episode][sporkful]:

> A while back we had legendary food critic Mimi Sheraton on the show. She told a story about a time she had 104 pastrami sandwiches in her car. She was taking them to a scale to weigh them to see which Jewish deli gave you the biggest sandwich for your money.
> 
> A Sporkful listener named Bill in San Clemente, California, heard that episode and called in with a pretty provocative question...
> 
> __Do places make smaller sandwiches for women?__

[sporkful]: http://www.sporkful.com/is-sandwich-sexism-real/

In [9]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import numpy as np
import pandas as pd
import pymc3 as pm
from pymc3.distributions.distribution import Bounded
from scipy import stats as st

from utils import plt, sns

In [2]:
# Sandwich data is in ounces...
cols = ["female", "deli", "sandwich"]
df = pd.DataFrame([
    # Male sandwiches
    [0, 1, 10.6],
    [0, 2, 22.2],
    [0, 3, 12.0],
    [0, 4, 11.7],
    # Female sandwiches
    [1, 1, 10.9],
    [1, 2, 25.1],
    [1, 3, 10.8],
    [1, 4, 13.7],
], columns=cols)
df.head()

,female,deli,sandwich
0,0,1,10.6
1,0,2,22.2
2,0,3,12.0
3,0,4,11.7
4,1,1,10.9


In [12]:
with pm.Model() as model:
    # Custom Distributions
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0, upper=100.0)
    
    # Priors
    intercept = pm.Normal("intercept", 0, 10)

    beta_mu = pm.Normal("beta_mu", 0, 10)
    beta_sigma = pm.HalfCauchy("beta_sigma", 2.5)  # Gelman 2006
    beta_nu = pm.Gamma("beta_nu", 2, 0.1)          # From the `Stan` docs
    beta = pm.StudentT("beta", nu=beta_nu, mu=beta_mu, sd=beta_sigma, shape=2)

    theta = intercept + beta[df.female.values]
    sigma = pm.HalfCauchy("sigma", 2.5)

    # Likelihood
    y = BoundedNormal("y", mu=theta, sd=sigma, observed=df.sandwich.values)

    # Sample
    trace = pm.sample(draws=5000, njobs=4, chain=4)

burn_in = 1000
trace = trace[burn_in:]

ValueError: Observed Bound distributions are not allowed. If you want to model truncated data you can use a pm.Potential in combination with the cumulative probability function. See pymc3/examples/censored_data.py for an example.

In [ ]:
print(pm.df_summary(trace))
pm.traceplot(trace)

In [ ]:
pm.plot_posterior(trace, point_estimate="median")

In [ ]:
pm.diagnostics.gelman_rubin(trace)

## Predictions

In [ ]:
ppc = pm.sample_ppc(trace, samples=2000, model=model)

In [ ]:
print(len(ppc["y"]))
ppc

In [ ]:
male = pd.Series(ppc["y"][:, :4].ravel())
female = pd.Series(ppc["y"][:, 4:].ravel())

print male.head()
print("\n")
print female.head()

In [ ]:
print(male.describe())
print(female.describe())

In [ ]:
sns.kdeplot(male, shade=True, label="Male")
sns.kdeplot(female, shade=True, label="Female")

## Conclusion

The distributions are almost exactly the same!